# Dependencies

In [1]:
import numpy as np

import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader

from sklearn.datasets import load_breast_cancer

# Dataset

$
x = 
\begin{bmatrix}
a_{1}^1 & a_{1}^2 & \cdots & a_{1}^n \\
a_{2}^1 & a_{2}^2 & \cdots & a_{2}^n \\
\vdots & \vdots & \ddots & \vdots \\
a_{m}^1 & a_{m}^2 & \cdots & a_{m}^n \\
\end{bmatrix}_{m \times n} \quad \text{(m: number of samples, n: number of features)}
$

$
y = 
\begin{bmatrix}
a_{1} \\
a_{2} \\
\vdots \\
a_{m} \\
\end{bmatrix}_{m \times 1} \quad \text{(m: number of samples)}
$

In [2]:
# load a dataset from sklearn.datasets
data = load_breast_cancer(return_X_y= True)

# properties of the dataset
num_samples, num_features  = data[0].shape
classes = np.unique(data[1])
num_classes = len(classes)

# log
for i, j in enumerate(['x', 'y']):
    print(f"{j} = data[{i}]")
    print(f"    -> data[{i}].shape: {eval(f'data[{i}].shape')}")
    print(f"    -> data[{i}].dtype: {eval(f'data[{i}].dtype')}")
print('-' * 50)
print(f"classes          : {classes}")
print(f"number of classes: {num_classes}")

x = data[0]
    -> data[0].shape: (569, 30)
    -> data[0].dtype: float64
y = data[1]
    -> data[1].shape: (569,)
    -> data[1].dtype: int32
--------------------------------------------------
classes          : [0 1]
number of classes: 2


# Torch Dataset
<ol>
    <li style="font-family: consolas;">Use TensorDataset &nbsp;&nbsp;&nbsp;: <span style="color: tomato">torch.utils.data.TensorDataset</span> does inherit from <span style="color: cyan">torch.utils.data.Dataset</span></li>
    <li style="font-family: consolas;">Create custom classes: custom classes also inherit from <span style="color: cyan">torch.utils.data.Dataset</span></li>
</ol>

### Custom Dataset

In [3]:
class CancerDataset(Dataset):
    def __init__(self, data) -> None:
        self.x = torch.from_numpy(data[0].astype(np.float32))
        self.y = torch.from_numpy(data[1].astype(np.float32)).view(-1, 1)
        self.n_samples = data[0].shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [4]:
# create an object of CancerDataset
dataset = CancerDataset(data)

# log
print(f"dataset.x.shape  : {dataset.x.shape}")
print(f"dataset.y.shape  : {dataset.y.shape}")
print('-' * 50)
print(f"first sample:")
print(f"    -> x: {dataset[0][0]}")
print(f"    -> y: {dataset[0][1]}")

dataset.x.shape  : torch.Size([569, 30])
dataset.y.shape  : torch.Size([569, 1])
--------------------------------------------------
first sample:
    -> x: tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
        8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01])
    -> y: tensor([0.])


### TensorDataset

In [5]:
# convert numpy.ndarray to torch.Tensor
data_x = torch.from_numpy(data[0].astype(np.float32))
data_y = torch.from_numpy(data[1].astype(np.float32)).view(-1, 1)

# create torch dataset
dataset = TensorDataset(data_x, data_y)

# log
print(f"dataset.tensors[0].shape  : {dataset.tensors[0].shape}")
print(f"dataset.tensors[1].shape  : {dataset.tensors[1].shape}")
print('-' * 50)
print(f"first sample:")
print(f"    -> x: {dataset[0][0]}")
print(f"    -> y: {dataset[0][1]}")

dataset.tensors[0].shape  : torch.Size([569, 30])
dataset.tensors[1].shape  : torch.Size([569, 1])
--------------------------------------------------
first sample:
    -> x: tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+00, 9.0530e-01,
        8.5890e+00, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01])
    -> y: tensor([0.])


# Torch DataLoader
<ul>
    <li>
        A DataLoader(<span style="font-family: consolas;color: tomato;">torch.utils.data.DataLoader</span>) is a utility that enables:
        <ul>
            <li>efficient loading datasets,</li>
            <li>handling batching,</li>
            <li>shuffling,</li>
            <li>parallel data loading</li>
        </ul>for training and evaluation in deep learning tasks.
    </li>
</ul>

### DataLoader for `Stochastic Gradient Descent`
   - the model updates its weights after processing each individual sample from the training dataset.
   - it is computationally efficient but can lead to noisy updates due to the variance in individual samples.

In [6]:
epochs = 2
batch_size = 1
dataloader = DataLoader(dataset, batch_size= batch_size, shuffle= True, num_workers= 2)

# log
for epoch in range(epochs):
    print(f"epoch {epoch}")
    for i, (x, y) in enumerate(dataloader):
        print(f"    iteration {i}")
        print(f"        x.shape: {x.shape}")
        print(f"        y.shape: {y.shape}")
        print("    weights are updated.\n")
    print(f"model saw the entire dataset")
    print('-' * 50)

epoch 0
    iteration 0
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 1
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 2
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 3
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 4
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 5
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 6
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 7
        x.shape: torch.Size([1, 30])
        y.shape: torch.Size([1, 1])
    weights are updated.

    iteration 8
        x.shape: torch.Size([1, 30])
        y.shape: to

### DataLoader for `Batch Gradient Descent`
   - the model updates its weights after processing the entire training dataset (all samples).
   - this method provides a more stable update direction, but it can be computationally expensive for large datasets.

In [7]:
epochs = 2
batch_size = dataset.tensors[0].shape[0]
dataloader = DataLoader(dataset, batch_size= batch_size, shuffle= True, num_workers= 2)

# log
for epoch in range(epochs):
    print(f"epoch {epoch}")
    for i, (x, y) in enumerate(dataloader):
        print(f"    iteration {i}")
        print(f"        x.shape: {x.shape}")
        print(f"        y.shape: {y.shape}")
        print("    weights are updated.\n")
    print(f"model saw the entire dataset")
    print('-' * 50)

epoch 0
    iteration 0
        x.shape: torch.Size([569, 30])
        y.shape: torch.Size([569, 1])
    weights are updated.

model saw the entire dataset
--------------------------------------------------
epoch 1
    iteration 0
        x.shape: torch.Size([569, 30])
        y.shape: torch.Size([569, 1])
    weights are updated.

model saw the entire dataset
--------------------------------------------------


### DataLoader for `Mini-Batch Gradient Descent`
   - the model updates its weights after processing a small batch of 'm' samples from the training dataset.
   - this method combines the advantages of both SGD and Batch Gradient Descent by providing a balance between efficiency and stability during training.

In [8]:
epochs = 2
batch_size = 4
dataloader = DataLoader(dataset, batch_size= batch_size, shuffle= True, num_workers= 2)

# log
for epoch in range(epochs):
    print(f"epoch {epoch}")
    for i, (x, y) in enumerate(dataloader):
        print(f"    iteration {i}")
        print(f"        x.shape: {x.shape}")
        print(f"        y.shape: {y.shape}")
        print("    weights are updated.\n")
    print(f"model saw the entire dataset")
    print('-' * 50)

epoch 0
    iteration 0
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 1
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 2
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 3
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 4
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 5
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 6
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 7
        x.shape: torch.Size([4, 30])
        y.shape: torch.Size([4, 1])
    weights are updated.

    iteration 8
        x.shape: torch.Size([4, 30])
        y.shape: to

# Create a custom model

In [9]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim) -> None:
        super(LogisticRegression, self).__init__()

        self.node = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        x = self.node(x)
        x = torch.sigmoid(x)
        return x

model = LogisticRegression(num_features, 1)
model

LogisticRegression(
  (node): Linear(in_features=30, out_features=1, bias=True)
)

# Train the model
   - Using `Mini-Batch Gradient Descent` technique.

In [10]:
# hyper-parameters
lr = 0.001
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= lr)
num_epochs = 20

In [11]:
model.train()
loss_per_epoch = []

for epoch in range(num_epochs):
    l = 0

    for x, y_true in dataloader:

        # forward
        y_pred = model(x)

        # backward
        loss = criterion(y_pred, y_true)
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()

        # update loss
        l += loss.item() * x.shape[0]
    
    loss_per_epoch.append(l / num_samples)
    
    print(f"epoch: {epoch:>2} - loss: {l:.5f}")

epoch:  0 - loss: 770.55640
epoch:  1 - loss: 640.39842
epoch:  2 - loss: 658.08175
epoch:  3 - loss: 559.39599
epoch:  4 - loss: 510.22692
epoch:  5 - loss: 519.08346
epoch:  6 - loss: 494.25869
epoch:  7 - loss: 483.25098
epoch:  8 - loss: 449.30475
epoch:  9 - loss: 393.17052
epoch: 10 - loss: 395.72004
epoch: 11 - loss: 391.08693
epoch: 12 - loss: 375.45738
epoch: 13 - loss: 351.05433
epoch: 14 - loss: 554.13332
epoch: 15 - loss: 545.82948
epoch: 16 - loss: 269.31821
epoch: 17 - loss: 359.02526
epoch: 18 - loss: 375.59177
epoch: 19 - loss: 270.08102


# Evaluate the model
   - Using the trainset to evaluate the model is only for educational purposes, and it's wrong!

In [12]:
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for x, true_y in dataloader:
        y_pred.extend((model(x) > 0.5).float())
        y_true.extend(true_y)

# log
for s in range(20):
    print(f"sample {s:>3}  ->  True class: {y_true[s].item()} | {y_pred[s].item()} :Predicted class")

sample   0  ->  True class: 0.0 | 0.0 :Predicted class
sample   1  ->  True class: 1.0 | 1.0 :Predicted class
sample   2  ->  True class: 0.0 | 0.0 :Predicted class
sample   3  ->  True class: 0.0 | 0.0 :Predicted class
sample   4  ->  True class: 1.0 | 1.0 :Predicted class
sample   5  ->  True class: 1.0 | 0.0 :Predicted class
sample   6  ->  True class: 1.0 | 1.0 :Predicted class
sample   7  ->  True class: 0.0 | 0.0 :Predicted class
sample   8  ->  True class: 0.0 | 0.0 :Predicted class
sample   9  ->  True class: 1.0 | 1.0 :Predicted class
sample  10  ->  True class: 1.0 | 1.0 :Predicted class
sample  11  ->  True class: 1.0 | 1.0 :Predicted class
sample  12  ->  True class: 0.0 | 0.0 :Predicted class
sample  13  ->  True class: 1.0 | 1.0 :Predicted class
sample  14  ->  True class: 1.0 | 1.0 :Predicted class
sample  15  ->  True class: 1.0 | 1.0 :Predicted class
sample  16  ->  True class: 1.0 | 1.0 :Predicted class
sample  17  ->  True class: 1.0 | 1.0 :Predicted class
sample  18